## Load libraries

In [2]:
import hmac
import hashlib
import time
import requests
from collections import OrderedDict
import json

## Get `auth_code`

When APP is authorized by the seller to access the seller's data.

Follow instruction [here](https://partner.tiktokshop.com/docv2/page/6632a7a926b40c02d97de61b) to get `auth_code`

## Get `access_token`

### Request

`https://auth.tiktok-shops.com/api/v2/token/get?app_key={input_your_app_key}&app_secret={input_your_app_secret}&auth_code={input_your_auth_code}&grant_type=authorized_code`

**Example:**

`https://auth.tiktok-shops.com/api/v2/token/get?app_key=6ef56li6i0q7q&app_secret=4e7d338hd5a000adae1fa1b3397b85cee97bfa06&auth_code=ROW_ypEGJBBBBBCxJaLyh8xhNQiz2th4E-3OfOfAEC7gGXmbzmYYxIdsLvWDUXLw71WEBcRDbgNWupAz4e4S9UOKNSceRR5pBWET0CUqFPoy4Fz5Csjy-nugZw&grant_type=authorized_code`

### Response

```

{"code":0,"message":"success","data":{"access_token":"ROW_XyuovxwBBBBlVdjoxXKfk6Wqsds1NFm_7ibCoZGrP_7lQiEnvyFYjshZzxIx00upkUdsqIcPtQn_oEKktjgjcrifi2IXQkMv60otQ6dz1q784D1j-kmXykq2V09hLWbTjyPfQHLgsWE","access_token_expire_in":1733122827,"refresh_token":"ROW_aobMIBBBBBBuFwi3woz_Ub428HyNqOCaCgMA5jZfAU_wyK3_ttQVjYoclR3IDJnTwd7FN-BVabg","refresh_token_expire_in":4854580847,"open_id":"IKb2JQBBBBDS_ZVUZ6C4O7YwD0KncLadCP8HRVFDQLqfhUR3HxIPBg","seller_name":"Example Seller","seller_base_region":"VN","user_type":0,"granted_scopes":["seller.authorization.info","serller.product.optimize","seller.product.basic","seller.order.info","seller.global_product.category.info","seller.global_product.info","seller.shop.info"]},"request_id":"20241125070026FF1E67884458C2058C1D"}

```


In [1]:
# Define access_token
access_token = 'ROW_XyuovxwBBBBlVdjoxXKfk6Wqsds1NFm_7ibCoZGrP_7lQiEnvyFYjshZzxIx00upkUdsqIcPtQn_oEKktjgjcrifi2IXQkMv60otQ6dz1q784D1j-kmXykq2V09hLWbTjyPfQHLgsWE' # for illustration purpose

## Create signature

Signature is used for authentication purpose. And its generation formular is instructed [here](https://partner.tiktokshop.com/doc/page/63fd743e715d622a338c4eab#Back%20To%20Top)

In [3]:
# Function to generate the HMAC-SHA256 signature
def generate_signature(path, query_params, secret):
    # Step 1: Sort the parameters alphabetically by key
    sorted_params = sorted(query_params.items())  # Returns a list of tuples (key, value)

    # Step 2: Concatenate the parameters in the format of {key}{value}
    parameter_str = path  # Start with the request path
    for key, value in sorted_params:
        parameter_str += f"{key}{value}"

    # Step 3: Wrap the concatenated string with the app secret
    signature_params = secret + parameter_str + secret

    # Step 4: Generate the HMAC-SHA256 signature
    signature = hmac.new(
        secret.encode("utf-8"),
        signature_params.encode("utf-8"),
        hashlib.sha256
    ).hexdigest()

    return signature

## Test call API

TiktokShop Partner API: [Link](https://partner.tiktokshop.com/docv2/page/6632a7434585a502e1cb5591)

I chose to test with `/api/products/details`. Deck: [Link](https://partner.tiktokshop.com/doc/page/64a2797f4b474b0283d38e4b?external_id=64a2797f4b474b0283d38e4b)

In [ ]:
# Predefined parameters in the correct order
path = "/api/products/details"
app_secret = "4e7d334f75a000adae1fa1b3397b85cee97bfa06" # for illustration purpose, input your own 'app_secret'

query_params = OrderedDict({
    "app_key": "6ef28ll0i0q7q", # for illustration purpose, input your own 'app_secret'
    "product_id": "1731027337055922770", # for illustration purpose, input your own 'product_id' to get data from
    "shop_cipher": "ROW_ytd-XgCCCCBXJ5tii038q0_1bHkVEj1f", # for illustration purpose, input your own 'shop_cipher'
    "shop_id": "7495222066017495634", # for illustration purpose, input your own 'shop_id'
    "timestamp": str(int(time.time())),  # to define dynamic timestamp
    "version": "202306"
})

# Generate the signature and add it to query_params
query_params["sign"] = generate_signature(path, query_params, app_secret)

# Build the final URL in the required order
base_url = "https://open-api.tiktokglobalshop.com"
full_url = f"{base_url}{path}?access_token={access_token}"
full_url += "&" + "&".join([f"{key}={value}" for key, value in query_params.items()])

# Headers
headers = {
    "x-tts-access-token": access_token
}

# Make the GET request
response = requests.get(full_url, headers=headers, verify=False)

# Print the response
print("Request URL:", full_url)
print("Response Status Code:", response.status_code)
print("Response JSON:", json.dumps(response.json(), indent=4, ensure_ascii=False))

**Illustration of successful response:**

```

Request URL: https://open-api.tiktokglobalshop.com/api/products/details?access_token=ROW_XyuovxwBBBBlVdjoxXKfk6Wqsds1NFm_7ibCoZGrP_7lQiEnvyFYjshZzxIx00upkUdsqIcPtQn_oEKktjgjcrifi2IXQkMv60otQ6dz1q784D1j-kmXykq2V09hLWbTjyPfQHLgsWE&app_key=6ef28ll0i0q7q&product_id=1731027337055922770&shop_cipher=ROW_ytd-XgCCCCBXJ5tii038q0_1bHkVEj1f&shop_id=7495222066017495634&timestamp=1732542297&version=202306&sign=bae1ebe89bae3a6e68031579658b270d9b1ec31b2804564b3dc3e508f4b2d488
Response Status Code: 200
Response JSON: {
    "code": 0,
    "data": {
        "brand": {
            "id": "2246979805606217474",
            "name": "Example brand",
            "status": 1
        },
        "category_list": [
            {
                "id": "722437",
                "is_leaf": false,
                "local_display_name": "Đồ ăn & Đồ uống",
                "parent_id": "0"
            },
            {
                "id": "224824",
                "is_leaf": false,
                "local_display_name": "Đồ uống",
                "parent_id": "700437"
            },
            {
                "id": "780612",
                "is_leaf": true,
                "local_display_name": "Cà phê",
                "parent_id": "700437"
            }
        ],
        "create_time": 1728536477222,
        "description": "<p style=\"text-align: left;\"><span><strong>HSD: 06/2026</strong></span></p><h2 style=\"text-align: left;\"><strong>Cà phê viên nén</strong></h2><img src=\"https://example-image-url.ibyteimg.com/tos-maliva-i-o3syd03w52-us/82fe91741a51473cbd3378fe8e0ba4cd~tplv-o3syd03w52-origin-jpeg.jpeg?dr=15568&nonce=43261&refresh_token=b803b7773001278cbb1203776104a94a&from=476444299&idc=maliva&ps=933b5bde&shcp=9b759fb9&shp=1c65f68b&t=555f072d\" width=\"2000\" height=\"2000\"><p>Cà phê viên nén Example Item Name là sự kết hợp tinh tế của những loại cà phê được lựa chọn kỹ càng, tạo nên một hỗn hợp vừa đủ nhưng cực kỳ lôi cuốn</p><p style=\"text-align: center;\"><span><br></span></p><p>Những điểm nhấn về hương thơm cũng là một trong những yếu tố khiến Example Item Name trở nên cuốn hút đối với những người yêu thích cà phê. Với mùi hương nồng nàn của trái cây khô đặc trưng, bạn sẽ cảm nhận được sự mời gọi tinh tế khi ngửi thấy hương thơm đầy quyến rũ lan tỏa từ tách cà phê. Không chỉ vậy, Example Item Name còn khiến bạn say đắm bởi hương vị thơm ngon đặc trưng của hạt phỉ. Mỗi lần thưởng thức, vị béo ngậy của hạt phỉ sẽ lan tỏa đều đặn trong khoang miệng, tạo nên một cảm giác bất ngờ và thú vị cho người thưởng thức. Đây chắc chắn là một trong những điểm nhấn độc đáo của Example Item Name, khiến cho cà phê này trở nên đáng để thử và tận hưởng.</p><p style=\"text-align: left;\"><span><strong>Thông tin chi tiết:</strong></span></p><ul><li>50% Robusta & 50% Arabica</li><li>Chất liệu nhôm</li><li>50 viên nén/hộp</li><li>Viên capsule dạng nén oxygen</li><li>Cùng định dạng với No brand</li><li>Sản xuất tại Ý</li></ul><p style=\"text-align: left;\"><span><strong>Đặc trưng:</strong></span></p><ul><li>Hàm lượng caffeine: Trung bình</li><li>Mùi hương: 4/5</li><li>Tính đầy đặn: 3/5</li><li>Độ ngọt: 3/5</li><li>Độ chua: 1/5</li><li>Hậu vị: 4/5</li></ul><p style=\"text-align: left;\"><span><br></span></p><h2 style=\"text-align: left;\"><strong>Cà phê viên nén Coffee Name Example Model - Đầy đặn, tinh tế</strong></h2><img src=\"https://example-image-url.com/tos-maliva-i-o3syd03w52-us/16cecea64ee947e095d89f312a473b38~tplv-o3syd03w52-origin-jpeg.jpeg?from=4250909378\" width=\"2000\" height=\"2000\"><p style=\"text-align: left;\"><span>Cà phê viên nén Example Model mang đến một sự pha trộn tinh tế nhằm nâng tầm hương vị cà phê truyền thống, chỉ có trong espresso Italia chính gốc. Hỗn hợp tinh tế này được pha trộn kỹ lưỡng, kết hợp giữa hạt Arabica ngọt ngào từ Trung Mỹ và loại hạt Santos tốt nhất từ Brazil, tạo nên một sự kết hợp hương vị tuyệt vời. Hơn nữa, thêm những hạt Robusta tinh tế từ Ấn Độ và Indonesia càng làm cho hỗn hợp trở nên đậm đà hơn, đồng thời khiến Example Model trở thành lựa chọn không thể chối từ cho những người yêu thích cà phê và muốn tận hưởng một trải nghiệm thú vị và đặc biệt.</span></p><p style=\"text-align: left;\"><span><br></span></p><p style=\"text-align: center;\"><span><br></span></p><p style=\"text-align: left;\"><span><strong>Thông tin chi tiết:</strong></span></p><ul><li>30% Robusta & 70% Arabica</li><li>Chất liệu nhôm</li><li>50 viên nén/hộp</li><li>Viên capsule dạng nén oxygen</li><li>Cùng định dạng với No brand</li><li>Sản xuất tại Ý</li></ul><p style=\"text-align: left;\"><span><strong>Đặc trưng:</strong></span></p><ul><li>Hàm lượng caffeine: Trung bình</li><li>Mùi hương: 4/5</li><li>Tính đầy đặn: 4/5</li><li>Độ ngọt: 4/5</li><li>Độ chua: 1/5</li><li>Hậu vị: 5/5</li></ul>",
        "images": [
            {
                "height": 1080,
                "id": "tos-maliva-i-o3syd03w52-us/2da4dea841ba4053b90e94cdde864a69",
                "thumb_url_list": [
                    "https://example-image-url.com/tos-maliva-i-o3syd03w52-us/2da4dea841ba4053b90e94cdde864a69~tplv-o3syd03w52-resize-jpeg:300:300.jpeg?from=520841845",
                    "https://example-image-url.ibyteimg.com/tos-maliva-i-o3syd03w52-us/2da4dea841ba4053b90e94cdde864a69~tplv-o3syd03w52-resize-jpeg:300:300.jpeg?from=520841845"
                ],
                "url_list": [
                    "https://example-image-url.com/tos-maliva-i-o3syd03w52-us/2da4dea841ba4053b90e94cdde864a69~tplv-o3syd03w52-origin-jpeg.jpeg?from=520841845",
                    "https://example-image-url.ibyteimg.com/tos-maliva-i-o3syd03w52-us/2da4dea841ba4053b90e94cdde864a69~tplv-o3syd03w52-origin-jpeg.jpeg?from=520841845"
                ],
                "width": 1080
            },
            {
                "height": 2000,
                "id": "tos-maliva-i-o3syd03w52-us/3ca7ee5cad0d4c1396d35be31b76e689",
                "thumb_url_list": [
                    "https://example-image-url.com/tos-maliva-i-o3syd03w52-us/3ca7ee5cad0d4c1396d35be31b76e689~tplv-o3syd03w52-resize-jpeg:300:300.jpeg?from=520841845",
                    "https://example-image-url.ibyteimg.com/tos-maliva-i-o3syd03w52-us/3ca7ee5cad0d4c1396d35be31b76e689~tplv-o3syd03w52-resize-jpeg:300:300.jpeg?from=520841845"
                ],
                "url_list": [
                    "https://example-image-url.com/tos-maliva-i-o3syd03w52-us/3ca7ee5cad0d4c1396d35be31b76e689~tplv-o3syd03w52-origin-jpeg.jpeg?from=520841845",
                    "https://example-image-url.ibyteimg.com/tos-maliva-i-o3syd03w52-us/3ca7ee5cad0d4c1396d35be31b76e689~tplv-o3syd03w52-origin-jpeg.jpeg?from=520841845"
                ],
                "width": 2000
            },
            {
                "height": 2000,
                "id": "tos-maliva-i-o3syd03w52-us/96ebfe4fdbed417ebe50a5fb43f79419",
                "thumb_url_list": [
                    "https://example-image-url.com/tos-maliva-i-o3syd03w52-us/96ebfe4fdbed417ebe50a5fb43f79419~tplv-o3syd03w52-resize-jpeg:300:300.jpeg?from=520841845",
                    "https://example-image-url.ibyteimg.com/tos-maliva-i-o3syd03w52-us/96ebfe4fdbed417ebe50a5fb43f79419~tplv-o3syd03w52-resize-jpeg:300:300.jpeg?from=520841845"
                ],
                "url_list": [
                    "https://example-image-url.com/tos-maliva-i-o3syd03w52-us/96ebfe4fdbed417ebe50a5fb43f79419~tplv-o3syd03w52-origin-jpeg.jpeg?from=520841845",
                    "https://example-image-url.ibyteimg.com/tos-maliva-i-o3syd03w52-us/96ebfe4fdbed417ebe50a5fb43f79419~tplv-o3syd03w52-origin-jpeg.jpeg?from=520841845"
                ],
                "width": 2000
            },
            {
                "height": 1024,
                "id": "tos-maliva-i-o3syd03w52-us/659f159e18014e019adb9b063cbdc700",
                "thumb_url_list": [
                    "https://example-image-url.com/tos-maliva-i-o3syd03w52-us/659f159e18014e019adb9b063cbdc700~tplv-o3syd03w52-resize-jpeg:300:300.jpeg?from=520841845",
                    "https://example-image-url.ibyteimg.com/tos-maliva-i-o3syd03w52-us/659f159e18014e019adb9b063cbdc700~tplv-o3syd03w52-resize-jpeg:300:300.jpeg?from=520841845"
                ],
                "url_list": [
                    "https://example-image-url.com/tos-maliva-i-o3syd03w52-us/659f159e18014e019adb9b063cbdc700~tplv-o3syd03w52-origin-jpeg.jpeg?from=520841845",
                    "https://example-image-url.ibyteimg.com/tos-maliva-i-o3syd03w52-us/659f159e18014e019adb9b063cbdc700~tplv-o3syd03w52-origin-jpeg.jpeg?from=520841845"
                ],
                "width": 1025
            },
            {
                "height": 1024,
                "id": "tos-maliva-i-o3syd03w52-us/85b3b8fb7d244044a3920a5ece8122fd",
                "thumb_url_list": [
                    "https://example-image-url.com/tos-maliva-i-o3syd03w52-us/85b3b8fb7d244044a3920a5ece8122fd~tplv-o3syd03w52-resize-jpeg:300:300.jpeg?from=520841845",
                    "https://example-image-url.ibyteimg.com/tos-maliva-i-o3syd03w52-us/85b3b8fb7d244044a3920a5ece8122fd~tplv-o3syd03w52-resize-jpeg:300:300.jpeg?from=520841845"
                ],
                "url_list": [
                    "https://example-image-url.com/tos-maliva-i-o3syd03w52-us/85b3b8fb7d244044a3920a5ece8122fd~tplv-o3syd03w52-origin-jpeg.jpeg?from=520841845",
                    "https://example-image-url.ibyteimg.com/tos-maliva-i-o3syd03w52-us/85b3b8fb7d244044a3920a5ece8122fd~tplv-o3syd03w52-origin-jpeg.jpeg?from=520841845"
                ],
                "width": 1024
            },
            {
                "height": 1440,
                "id": "tos-maliva-i-o3syd03w52-us/47958ef469dd49cdbe4000bdfde98648",
                "thumb_url_list": [
                    "https://example-image-url.com/tos-maliva-i-o3syd03w52-us/47958ef469dd49cdbe4000bdfde98648~tplv-o3syd03w52-resize-jpeg:300:300.jpeg?from=520841845",
                    "https://example-image-url.ibyteimg.com/tos-maliva-i-o3syd03w52-us/47958ef469dd49cdbe4000bdfde98648~tplv-o3syd03w52-resize-jpeg:300:300.jpeg?from=520841845"
                ],
                "url_list": [
                    "https://example-image-url.com/tos-maliva-i-o3syd03w52-us/47958ef469dd49cdbe4000bdfde98648~tplv-o3syd03w52-origin-jpeg.jpeg?from=520841845",
                    "https://example-image-url.ibyteimg.com/tos-maliva-i-o3syd03w52-us/47958ef469dd49cdbe4000bdfde98648~tplv-o3syd03w52-origin-jpeg.jpeg?from=520841845"
                ],
                "width": 1440
            }
        ],
        "is_cod_open": true,
        "package_height": 17,
        "package_length": 16,
        "package_weight": "0.5",
        "package_width": 15,
        "product_attributes": [
            {
                "id": "100347",
                "name": "Số lượng trên mỗi gói",
                "values": [
                    {
                        "id": "6945370083732670210",
                        "name": "50"
                    }
                ]
            },
            {
                "id": "100346",
                "name": "Thành phần",
                "values": [
                    {
                        "id": "7084232731210548998",
                        "name": "Bột cà phê"
                    }
                ]
            },
            {
                "id": "100481",
                "name": "Loại bao bì",
                "values": [
                    {
                        "id": "1001690",
                        "name": "Hộp nhỏ"
                    }
                ]
            },
            {
                "id": "100486",
                "name": "Cà phê rang",
                "values": [
                    {
                        "id": "1000907",
                        "name": "Trung bình"
                    }
                ]
            },
            {
                "id": "100485",
                "name": "Loại cà phê",
                "values": [
                    {
                        "id": "1001789",
                        "name": "Cà phê viên"
                    }
                ]
            },
            {
                "id": "100704",
                "name": "Hương vị",
                "values": [
                    {
                        "id": "7070029559391012614",
                        "name": "Cà phê"
                    }
                ]
            },
            {
                "id": "100336",
                "name": "Khu vực xuất xứ",
                "values": [
                    {
                        "id": "7072976490182395654",
                        "name": "Ý"
                    }
                ]
            },
            {
                "id": "100337",
                "name": "Hạn sử dụng",
                "values": [
                    {
                        "id": "1000662",
                        "name": "24 tháng"
                    }
                ]
            },
            {
                "id": "100702",
                "name": "Loại bảo quản",
                "values": [
                    {
                        "id": "1001585",
                        "name": "Tủ đựng chén"
                    }
                ]
            }
        ],
        "product_id": "1731029337055922770",
        "product_name": "Hộp 50 viên cà phê viên nén nhôm Coffee Name - Example Model - Example Item Name (Aluminum Example Model - Example Item Name Capsules) - Nhập khẩu từ Ý - Tương thích với máy capsule No brand Coffee - Cafe",
        "product_status": 4,
        "skus": [
            {
                "id": "1731159324007369298",
                "price": {
                    "currency": "VND",
                    "original_price": "200800"
                },
                "sales_attributes": [
                    {
                        "id": "100092",
                        "name": "Hương vị",
                        "sku_img": {
                            "height": 2000,
                            "id": "tos-maliva-i-o3syd03w52-us/438d8476784d4a5bb75108445cb46d97",
                            "thumb_url_list": [
                                "https://example-image-url.com/tos-maliva-i-o3syd03w52-us/438d8476784d4a5bb75108445cb46d97~tplv-o3syd03w52-resize-jpeg:300:300.jpeg?from=520841845",
                                "https://example-image-url.ibyteimg.com/tos-maliva-i-o3syd03w52-us/438d8476784d4a5bb75108445cb46d97~tplv-o3syd03w52-resize-jpeg:300:300.jpeg?from=520841845"
                            ],
                            "url_list": [
                                "https://example-image-url.com/tos-maliva-i-o3syd03w52-us/438d8476784d4a5bb75108445cb46d97~tplv-o3syd03w52-origin-jpeg.jpeg?from=520841845",
                                "https://example-image-url.ibyteimg.com/tos-maliva-i-o3syd03w52-us/438d8476784d4a5bb75108445cb46d97~tplv-o3syd03w52-origin-jpeg.jpeg?from=520841845"
                            ],
                            "width": 2000
                        },
                        "value_id": "7359773681217783553",
                        "value_name": "Example model"
                    }
                ],
                "seller_sku": "BBAINTE50",
                "stock_infos": [
                    {
                        "available_stock": 17,
                        "warehouse_id": "7319540688915072774"
                    }
                ]
            },
            {
                "id": "1731159324007434834",
                "price": {
                    "currency": "VND",
                    "original_price": "200800"
                },
                "sales_attributes": [
                    {
                        "id": "100092",
                        "name": "Hương vị",
                        "sku_img": {
                            "height": 2000,
                            "id": "tos-maliva-i-o3syd03w52-us/f40aff7033d84bcbb34a5083a9df3b03",
                            "thumb_url_list": [
                                "https://example-image-url.com/tos-maliva-i-o3syd03w52-us/f40aff7033d84bcbb34a5083a9df3b03~tplv-o3syd03w52-resize-jpeg:300:300.jpeg?from=520841845",
                                "https://example-image-url.ibyteimg.com/tos-maliva-i-o3syd03w52-us/f40aff7033d84bcbb34a5083a9df3b03~tplv-o3syd03w52-resize-jpeg:300:300.jpeg?from=520841845"
                            ],
                            "url_list": [
                                "https://example-image-url.com/tos-maliva-i-o3syd03w52-us/f40aff7033d84bcbb34a5083a9df3b03~tplv-o3syd03w52-origin-jpeg.jpeg?from=520841845",
                                "https://example-image-url.ibyteimg.com/tos-maliva-i-o3syd03w52-us/f40aff7033d84bcbb34a5083a9df3b03~tplv-o3syd03w52-origin-jpeg.jpeg?from=520841845"
                            ],
                            "width": 2000
                        },
                        "value_id": "7131065758049928966",
                        "value_name": "Example Model"
                    }
                ],
                "seller_sku": "BBAGRAC50",
                "stock_infos": [
                    {
                        "available_stock": 18,
                        "warehouse_id": "7319540688915072774"
                    }
                ]
            }
        ],
        "update_time": 1732501437474
    },
    "message": "Success",
    "request_id": "20241125134513A9BEC419DED58027F622"
}

```